## Notebook used to generate a fake supplier dataset and create the recommandation module. 

### Dataset recommandation fournisseur

In [52]:
import pandas as pd
import numpy as np
import random
import time
import math
import geopy.distance

In [53]:
import warnings
warnings.filterwarnings("ignore")

#### Créer les colonnes du datset

In [54]:
df = pd.DataFrame([])

Répartition des fermes: 25 fermes d'élevage, 100 de fruits et légumes, 75 de céréals

In [55]:
from random import randint

noms_fermes = pd.DataFrame(np.zeros(200))

#fermes d'élevge
noms_fermes.iloc[:10] = ['Les produits laitiers de Dédé', 'Beaux bauvins', 'New Ranch', 'Oh la vache', 'La Rave des Bêtes', 'Le toro', 'Les Cochonets', 'La chèvre de Monsieur', 'Les poules cools', 'La main à la pâte']

nom = ['Le Cochon', 'Le Troupeau', 'Le Poulailler', 'Le Poulet', 'Le Bauvin', 'Le coq', 'Le toro', 'Le champs', 'Le lait', 'Escargot']
adjectif = ['Vert', 'Blanc', 'Bavard', 'Fêtard', 'Détendu', 'Abrité', 'Ensoleillé', 'Heureux', 'Joli', 'Bleu', 'de Jean-Mi', 'du coin', 'régional', 'national', 'de qualité', 'de Bretagne', 'de Normandie', 'en folie', 'à la ferme', 'qui chante', 'qui danse']

def inventeur_de_nom(noms_fermes, nom, adjectif, nb_noms, debut):
    for i in range(nb_noms):
        n = randint(0, len(nom)-1)
        m = randint(0, len(adjectif)-1)
        noms_fermes.iloc[i+debut,0] = nom[n] + ' ' + adjectif[m]
    return noms_fermes

noms_fermes = inventeur_de_nom(noms_fermes, nom, adjectif, 40, 10)


In [56]:
#fermes de fruits et légumes
noms_fermes.iloc[50:60, 0] = ['La belle ferme', 'Les produits de mamie', 'Fruits et légumes biologiques et écologiques', 
                            'La Maison verte', 'La New Récolte', 'Le Cœur vert', 'La saison de la moisson', 'Soleil et pluie', 
                            'Les cueillettes', 'Dans les bois']

In [57]:
nom = ['La Paille', 'La rosée', 'La main', 'La tournée', 'La région', 'Perspective', 'Recherche', 'La cabane', 'La maison', 'La variété', 'La collection']
adjectif = ['Verte', 'Bio', 'Bio', 'de saison', 'Fraîche', 'Pour la vie', 'depuis toujours', 'certifiée', 'honnête', 'sublime']

noms_fermes = inventeur_de_nom(noms_fermes, nom, adjectif, 80, 60)

In [58]:
import math

def definisseur_de_nom(noms_fermes, nom, adjectif, nb_noms, debut):
    for i in range(3):
        for j in range(19):
            noms_fermes.iloc[i*19+j+debut, 0] = nom[i] + ' ' + adjectif[j]
        noms_fermes.iloc[3*19+i+debut, 0] = nom[i] + ' ' + adjectif[i]
    
    return noms_fermes

In [59]:
#fermes de céréals
nom = ['La Ferme de', 'Exploitation de', 'La culture de', 'Le blé de']
proprio = ['Pauline', 'M. Alain', 'Jean Pierre', 'M. Bec', 'Merie Brenon', 'Mme Paul Gauthier', 'M. Yves Laurent', 
        'Saint Romain', 'M. Becquart', 'M. Dubec', 'Jeanne', 'Thomas Berger', 'Paul Artisan', 
        'Mme Line Maizeray', 'Avondance', 'Louise Puisenval', 'M. Jean Pres', 'M. Christian', 'M. Pierre Mas', 'Mme Saulcy']

noms_fermes = definisseur_de_nom(noms_fermes, nom, proprio, 60, 140)

In [60]:
df['Nom de la ferme'] = noms_fermes

In [61]:
df['Nom de la ferme']

0      Les produits laitiers de Dédé
1                      Beaux bauvins
2                          New Ranch
3                        Oh la vache
4                  La Rave des Bêtes
                   ...              
195       La culture de M. Christian
196      La culture de M. Pierre Mas
197              La Ferme de Pauline
198         Exploitation de M. Alain
199        La culture de Jean Pierre
Name: Nom de la ferme, Length: 200, dtype: object

In [62]:
df['Produit issu de la ferme'] = np.zeros((200,))
df['Produit issu de la ferme'][:60] = ['parmesan']*60
df['Produit issu de la ferme'][60:140] = ['fraise']*80
df['Produit issu de la ferme'][140:] = ['champignon']*60

##### Ville et localisation de la ferme

In [63]:
villes = pd.read_csv("data/villes_fr.csv")
villes= villes.drop_duplicates(subset ="city", inplace = False)

In [64]:
villes.capital.value_counts(dropna=False)

NaN        9189
minor       303
admin        12
primary       1
Name: capital, dtype: int64

In [65]:
# Localistaion de la ferme
temp = villes[~villes.capital.isin(["minor","primary","admin"])]
villes_fermes = random.sample(list(temp.city),200)
df['city'] = villes_fermes
temp = villes[["city","lat","lng"]]
df = pd.merge(df,temp, on='city')

##### Surface

In [66]:
df['Surface'] = np.random.choice(range(10, 150),len(df),replace=True)

##### Emmission du mode de transport utilisé (en g CO2 / km)

In [67]:
# random entre 400 et 600 g CO2 / km
df['Emmission transport'] = np.random.randint(400, 600, size=(200,))

##### Emmision du packaging de transport utilisé (en g CO2 / kg)

In [68]:
df_packaging = pd.read_csv('data/emballages.csv')

In [69]:
df_packaging.head()

,Catégorie,Matière,kg équivalent CO2 / tonne,Taux de recyclage,Id matière,Recyclé,Type emballage
0,Papier,Pâte à papier primaire,297.0,62.0,1,0,Pochette
1,Papier,Pâte à papier primaire recyclée,317.0,52.0,2,1,Pochette
2,Papier,Pâte à papier Eucalyptus,440.0,61.5,3,0,Pochette
3,Papier,Pâte à papier Résineux,410.0,56.5,4,0,Pochette
4,Papier,Pâte à papier Feuillus,400.0,60.5,5,0,Pochette


In [70]:
df_packaging['Catégorie'].unique()

array(['Papier', 'Carton', 'Plastique', 'Polystyrène', 'PVC', 'Nylon',
       'Verre', 'Bois'], dtype=object)

In [71]:
df['Type packaging'] = np.zeros((200,))
df['Emmission type packaging'] = np.zeros((200,))
df['Masse packaging'] = np.zeros((200,))

In [72]:
# faire quelques recherches sur range de random
sub_mat = df_packaging[df_packaging.Catégorie.isin(['Plastique', 'Carton', 'Polystyrène', 'Plastique'])]

for i in range(200):
    type = np.random.randint(0, 10)
    df['Type packaging'][i] = sub_mat.iloc[type,1]
    df['Emmission type packaging'][i] = sub_mat.iloc[type,2]
    df['Masse packaging'][i] = np.random.random()*0.01 + 0.2
    

In [73]:
df

,Nom de la ferme,Produit issu de la ferme,city,lat,lng,Surface,Emmission transport,Type packaging,Emmission type packaging,Masse packaging
0,Les produits laitiers de Dédé,parmesan,Saleilles,42.6542,2.9517,126,492,Carton recyclé,670.0,0.206003
1,Beaux bauvins,parmesan,La Môle,43.2083,6.4656,48,532,Carton recyclé,670.0,0.202746
2,New Ranch,parmesan,Saint-Fuscien,49.8375,2.3150,113,596,Carton recyclé,670.0,0.206627
3,Oh la vache,parmesan,Saint-Vit,47.1814,5.8108,101,439,Carton primaire,390.0,0.203399
4,La Rave des Bêtes,parmesan,Zuydcoote,51.0642,2.4894,53,449,Carton primaire,390.0,0.208787
...,...,...,...,...,...,...,...,...,...,...
195,La culture de M. Christian,champignon,Fumay,49.9936,4.7069,139,598,Carton recyclé,670.0,0.208385
196,La culture de M. Pierre Mas,champignon,Martignat,46.2094,5.6089,51,571,Carton primaire,390.0,0.201824
197,La Ferme de Pauline,champignon,Modane,45.2014,6.6733,17,595,Carton primaire,390.0,0.207588
198,Exploitation de M. Alain,champignon,Saint-Jeoire,46.1383,6.4594,40,485,Carton recyclé,670.0,0.203017


##### Label bas carbone, bio, produit fermier et AOP (Appelation Origine Protégée)

In [74]:
# random de 0 (non) et 1 (oui)
df['Label bas carbone'] = np.random.randint(0, 2, size=(200,))

In [75]:
# random de 0 (non) et 1 (oui)
df['Label bio'] = np.random.randint(0, 2, size=(200,))

In [76]:
# random de 0 (non) et 1 (oui)
df['Label produit fermier'] = np.random.randint(0, 2, size=(200,))

In [77]:
# random de 0 (non) et 1 (oui)
df['Label aop'] = np.random.randint(0, 2, size=(200,))


In [78]:
df.head()

,Nom de la ferme,Produit issu de la ferme,city,lat,lng,Surface,Emmission transport,Type packaging,Emmission type packaging,Masse packaging,Label bas carbone,Label bio,Label produit fermier,Label aop
0,Les produits laitiers de Dédé,parmesan,Saleilles,42.6542,2.9517,126,492,Carton recyclé,670.0,0.206003,0,1,0,0
1,Beaux bauvins,parmesan,La Môle,43.2083,6.4656,48,532,Carton recyclé,670.0,0.202746,1,0,0,0
2,New Ranch,parmesan,Saint-Fuscien,49.8375,2.3150,113,596,Carton recyclé,670.0,0.206627,0,1,0,1
3,Oh la vache,parmesan,Saint-Vit,47.1814,5.8108,101,439,Carton primaire,390.0,0.203399,1,0,0,1
4,La Rave des Bêtes,parmesan,Zuydcoote,51.0642,2.4894,53,449,Carton primaire,390.0,0.208787,0,1,1,0


In [79]:
villes.iloc[3,0]

'Marseille'

### Example

In [80]:
ville_acheteur = 'Lyon'
nom_ferme = 'New Ranch'
produit = 'champignon'

##### Distance parcourue jusque production (en km)

In [81]:
# calcul de la distance 
def calcul_distance(data, villes, ville_acheteur):
    temp = villes[villes.city == ville_acheteur].reset_index(drop=True)
    data['Distance'] = list(zip(data.lat, data.lng))
    data['Distance'] = data['Distance'].apply(lambda x : geopy.distance.geodesic(x, (temp['lat'][0],temp['lng'][0])).km)
    return df


#### Créer le score métric du dataset 

In [82]:
x_0 = df[df['Nom de la ferme'] == nom_ferme]

In [83]:
df.columns

Index(['Nom de la ferme', 'Produit issu de la ferme', 'city', 'lat', 'lng',
       'Surface', 'Emmission transport', 'Type packaging',
       'Emmission type packaging', 'Masse packaging', 'Label bas carbone',
       'Label bio', 'Label produit fermier', 'Label aop'],
      dtype='object')

In [84]:
def calcul_score(df, villes, ville_acheteur, produit, nom_ferme):
    
    df = calcul_distance(df, villes, ville_acheteur)
    df['Ville acheteur'] = [ville_acheteur]*200
    
    #définir la ferme et le produit par rapport  auquel on calcule le score
    x_0 = df[df['Nom de la ferme'] == nom_ferme].reset_index(drop=True)
    
    #choisir fermes qui produisent le meme produit uniquement
    sub_df = df[(df['Produit issu de la ferme'] == produit) 
                & (df['Label bio'] == x_0['Label bio'][0])]

    num_columns = ['Surface', 'Emmission transport', 'Emmission type packaging', 'Masse packaging', 'Distance']
    cat_columns = ['Label bas carbone', 'Label bio', 'Label produit fermier', 'Label aop']
    
    delta_num = pd.DataFrame([])
    for c in num_columns:
        #mesurer les deltas en pourcentage pour les colonnes numériques
        delta_num[c] = sub_df[c].apply(lambda x: (x - x_0[c])/x_0[c])

    delta_cat = pd.DataFrame([])
    for c in cat_columns:
        #mesurer les deltas net pour les colonnes catégoriques (0 et 1 des labels)
        delta_cat[c] = sub_df[c].apply(lambda x: (x - x_0[c]))
    
    sub_df['CO2'] = sub_df['Distance'].multiply(sub_df['Emmission transport'])+sub_df['Emmission type packaging'].multiply(sub_df['Masse packaging'])
    
    x_0['CO2'] = x_0['Distance']*x_0['Emmission transport']+x_0['Emmission type packaging']*x_0['Masse packaging']
    
    sub_df['% CO2 reco vs init'] = 1 + sub_df['CO2'].apply(lambda x: (x - x_0['CO2'])/x_0['CO2'])
    sub_df['score carbon'] = sub_df['CO2'].apply(lambda x: (x- min(sub_df['CO2']))/(max(sub_df['CO2'])-min(sub_df['CO2'])))                                                                                                                         
    sub_df['score fournisseur'] = 0.7 - ((sub_df['score carbon']) - 0.2 * sub_df['Label bas carbone'] - 0.1 * sub_df['Label aop'])
    
    sub_df = sub_df.sort_values('score fournisseur', ascending=True)
    
    return sub_df


In [85]:
sub_df = calcul_score(df, villes, ville_acheteur, produit, nom_ferme)

In [86]:
sub_df.head()

,Nom de la ferme,Produit issu de la ferme,city,lat,lng,Surface,Emmission transport,Type packaging,Emmission type packaging,Masse packaging,Label bas carbone,Label bio,Label produit fermier,Label aop,Distance,Ville acheteur,CO2,% CO2 reco vs init,score carbon,score fournisseur
159,Exploitation de Pauline,champignon,Callac,48.4042,-3.4278,61,581,Carton primaire,390.0,0.209310,0,1,1,0,692.770864,Lyon,402581.502877,1.374588,1.000000,-0.300000
152,La Ferme de Paul Artisan,champignon,Saint-Martin-Boulogne,50.7258,1.6322,39,560,Plastique blanc,2025.0,0.208550,0,1,1,0,601.234198,Lyon,337113.463903,1.151051,0.834290,-0.134290
179,La culture de M. Alain,champignon,Quiberon,47.4833,-3.1206,50,408,Plastique blanc,2025.0,0.208126,0,1,1,0,638.755512,Lyon,261033.705091,0.891282,0.641721,0.058279
158,La Ferme de M. Pierre Mas,champignon,Ménéac,48.1394,-2.4608,143,541,Carton primaire,390.0,0.204597,1,1,0,0,615.184538,Lyon,332894.627937,1.136646,0.823612,0.076388
183,La culture de Mme Paul Gauthier,champignon,Vatteville,49.5000,0.6833,104,556,Plastique blanc,2025.0,0.207836,0,1,0,1,519.927776,Lyon,289500.710552,0.988481,0.713775,0.086225


In [87]:
def calcul_score_sans_ferme(data, villes, ville_acheteur, ingredient):
    temp = data.copy()
    
    #temp['Ville acheteur'] = [ville_acheteur]*200
    
    temp = temp[temp['Produit issu de la ferme'] == ingredient].reset_index(drop=True)
    
    temp['CO2'] = temp['Distance'].multiply(temp['Emmission transport'])+temp['Emmission type packaging'].multiply(temp['Masse packaging'])
    
    temp['score carbon'] = temp['CO2'].apply(lambda x: (x- min(temp['CO2']))/(max(temp['CO2'])-min(temp['CO2'])))                                                                                                                         
    temp['score fournisseur'] = 0.7 - ((temp['score carbon']) - 0.2 * temp['Label bas carbone'] - 0.1 * temp['Label aop'])
    
    reco = temp.sort_values('score fournisseur', ascending = False)
    
    return reco

In [88]:
df = calcul_distance(df, villes, 'Nantes')

In [89]:
reco = calcul_score_sans_ferme(df, villes, 'Nantes', 'fraise')
reco.iloc[:10,:]

,Nom de la ferme,Produit issu de la ferme,city,lat,lng,Surface,Emmission transport,Type packaging,Emmission type packaging,Masse packaging,Label bas carbone,Label bio,Label produit fermier,Label aop,Distance,Ville acheteur,CO2,score carbon,score fournisseur
75,La main Bio,fraise,Toutlemonde,47.0550,-0.7669,124,590,Carton primaire,390.0,0.200722,1,1,0,1,62.315986,Lyon,36844.713316,0.064751,0.935249
71,La région sublime,fraise,Les Herbiers,46.8711,-1.0136,130,484,Carton primaire,390.0,0.200313,1,1,0,0,56.276948,Lyon,27316.164691,0.042130,0.857870
76,Recherche Fraîche,fraise,Héric,47.4125,-1.6517,94,415,Carton primaire,390.0,0.201414,0,1,0,1,22.869789,Lyon,9569.514072,0.000000,0.800000
21,La rosée Bio,fraise,Gainneville,49.5178,0.2511,77,421,Carton primaire,390.0,0.207645,1,1,0,1,288.521879,Lyon,121548.692836,0.265837,0.734163
72,La région Verte,fraise,Longeville-sur-Mer,46.4247,-1.4887,75,546,Carton recyclé,670.0,0.201475,0,0,1,1,88.335701,Lyon,48366.281177,0.092103,0.707897
49,La collection sublime,fraise,Blainville-sur-Orne,49.2289,-0.3000,143,409,Carton recyclé,670.0,0.202835,1,0,1,0,242.184181,Lyon,99189.229240,0.212756,0.687244
60,La cabane Bio,fraise,Abondant,48.7847,1.4392,86,547,Carton primaire,390.0,0.200838,1,0,1,1,283.147295,Lyon,154959.896973,0.345154,0.654846
2,Recherche Bio,fraise,Pleumeur-Bodou,48.7731,-3.5178,14,565,Carton primaire,390.0,0.209011,1,0,1,0,226.702253,Lyon,128168.287256,0.281551,0.618449
51,La tournée Verte,fraise,Laroque-Timbaut,44.2831,0.7619,142,457,Carton recyclé,670.0,0.205570,1,0,0,1,372.589881,Lyon,170411.307377,0.381835,0.618165
45,La rosée depuis toujours,fraise,Boutigny-sur-Essonnes,48.4358,2.3775,67,526,Plastique transparent,2350.0,0.202563,1,0,0,1,323.879128,Lyon,170836.443313,0.382845,0.617155


In [90]:
def recommender_fournisseur(data, villes, ville_acheteur, produit, nom_ferme = None, metric = 'score fournisseur'):
    
    if nom_ferme is None : 
        df_resultat = calcul_score_sans_ferme(data, villes, ville_acheteur, produit).sort_values(by = metric, ascending = False)
        resultat = df_resultat.head(10)
    else : 
        df_resultat = calcul_score(data, villes, ville_acheteur, produit).sort_values(by = metric, ascending = False)
        resultat = df_resultat.head(10)
    
    return resultat

In [91]:
recommender_fournisseur(df, villes, 'Paris', 'fraise')

,Nom de la ferme,Produit issu de la ferme,city,lat,lng,Surface,Emmission transport,Type packaging,Emmission type packaging,Masse packaging,Label bas carbone,Label bio,Label produit fermier,Label aop,Distance,Ville acheteur,CO2,score carbon,score fournisseur
75,La main Bio,fraise,Toutlemonde,47.0550,-0.7669,124,590,Carton primaire,390.0,0.200722,1,1,0,1,62.315986,Lyon,36844.713316,0.064751,0.935249
71,La région sublime,fraise,Les Herbiers,46.8711,-1.0136,130,484,Carton primaire,390.0,0.200313,1,1,0,0,56.276948,Lyon,27316.164691,0.042130,0.857870
76,Recherche Fraîche,fraise,Héric,47.4125,-1.6517,94,415,Carton primaire,390.0,0.201414,0,1,0,1,22.869789,Lyon,9569.514072,0.000000,0.800000
21,La rosée Bio,fraise,Gainneville,49.5178,0.2511,77,421,Carton primaire,390.0,0.207645,1,1,0,1,288.521879,Lyon,121548.692836,0.265837,0.734163
72,La région Verte,fraise,Longeville-sur-Mer,46.4247,-1.4887,75,546,Carton recyclé,670.0,0.201475,0,0,1,1,88.335701,Lyon,48366.281177,0.092103,0.707897
49,La collection sublime,fraise,Blainville-sur-Orne,49.2289,-0.3000,143,409,Carton recyclé,670.0,0.202835,1,0,1,0,242.184181,Lyon,99189.229240,0.212756,0.687244
60,La cabane Bio,fraise,Abondant,48.7847,1.4392,86,547,Carton primaire,390.0,0.200838,1,0,1,1,283.147295,Lyon,154959.896973,0.345154,0.654846
2,Recherche Bio,fraise,Pleumeur-Bodou,48.7731,-3.5178,14,565,Carton primaire,390.0,0.209011,1,0,1,0,226.702253,Lyon,128168.287256,0.281551,0.618449
51,La tournée Verte,fraise,Laroque-Timbaut,44.2831,0.7619,142,457,Carton recyclé,670.0,0.205570,1,0,0,1,372.589881,Lyon,170411.307377,0.381835,0.618165
45,La rosée depuis toujours,fraise,Boutigny-sur-Essonnes,48.4358,2.3775,67,526,Plastique transparent,2350.0,0.202563,1,0,0,1,323.879128,Lyon,170836.443313,0.382845,0.617155
